###  Dynasty risk adjustment

In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import pandas as pd
import numpy as np
from pathlib import Path
from TPT_generator_python import DataBucket, TPTFetcher, TPTGenerator

In [2]:
DATE = "2020-12-31"
CLIENT = "Pictet"
ISIN = "LU1787059465"
SYMADJ = -0.48
SOURCE_DIR = Path("../data")
OUTPUT_DIR = './production/example'
ALL = slice(None)

In [3]:
generator = TPTGenerator(DATE,
                         CLIENT,
                         OUTPUT_DIR,
                         SOURCE_DIR,
                         sym_adj=SYMADJ,
                         shareclass_isin=ISIN)
generator


    Reporting date:          2020-12-31
    Client:                  Pictet
    Shareclass isin:         LU1787059465 
    number of instruments:   82
        

In [6]:
generator.data_bucket.get_instruments(ALL).shape
generator.data_bucket.get_groups()

0     20
11    28
14    56
Name: group_id, dtype: int64

In [5]:
print("subfund currency: ", generator.data_bucket.get_subfund_infos('subfund_currency'))
generator.data_bucket.get_shareclass_infos(isin=ALL)

subfund currency:  USD


,id,shareclass,shareclass_id,shareclass_currency,shareclass_name,id_subfund,type_tpt
code_isin,,,,,,,
LU1787059465,463,A1E,None,EUR,Decalia SICAV - Circular Economy - A1E,148,1
LU1787059036,464,A1U,None,USD,Decalia SICAV - Circular Economy - A1U,148,1
LU1787061107,465,I,None,USD,Decalia SICAV - Circular Economy - I,148,1
LU1787061362,466,IC,None,CHF,Decalia SICAV - Circular Economy - IC,148,1
LU1787061529,467,IE,None,EUR,Decalia SICAV - Circular Economy - IE,148,1
LU1787060471,468,R,None,USD,Decalia SICAV - Circular Economy - R,148,1


In [6]:
navs = generator.data_bucket.get_shareclass_nav(isin=ALL, info=['shareclass_total_net_asset_sc_curr',
                                                                'shareclass_total_net_asset_sf_curr',
                                                                'subfund_total_net_asset'])
navs

,shareclass_total_net_asset_sc_curr,shareclass_total_net_asset_sf_curr,subfund_total_net_asset
shareclass,,,
LU1280365476,2.580789e+07,2.386180e+07,3.301555e+08
LU1280365633,1.331171e+08,1.331171e+08,3.301555e+08
LU1280365393,1.032699e+08,1.032699e+08,3.301555e+08
LU1280365559,1.648996e+07,1.347714e+07,3.301555e+08
LU1483663818,4.054038e+07,3.748335e+07,3.301555e+08
LU1840818220,1.094834e+06,1.223149e+06,3.301555e+08
LU1586705938,7.134391e+06,5.830894e+06,3.301555e+08
LU1508332993,1.189213e+07,1.189213e+07,3.301555e+08


In [7]:
print("sum of shareclass navs: ", navs["shareclass_total_net_asset_sf_curr"].sum())
print("TNA of subfund:         ", navs["subfund_total_net_asset"].iloc[0])
## if sum == TNA ==> no navs missing

sum of shareclass navs:  330155476.93
TNA of subfund:          330155476.93


In [27]:
generator.data_bucket.processing_data = None
dist_vec = generator.data_bucket.get_distribution_vector()
print(dist_vec.sum())
## if no fake line, should be equel to shareclass nav
print(generator.data_bucket.get_shareclass_nav(info=['shareclass_total_net_asset_sf_curr']).iloc[0])

23861795.520000003
23861795.52


In [12]:
sum_ = generator.data_bucket.processing_data.loc[(ALL, ALL), "distribution"].sum()
sum_1 = generator.data_bucket.processing_data.loc[(ALL, navs.index[0]), "distribution"].sum()
sum_2 = generator.data_bucket.processing_data.loc[(ALL, navs.index[1]), "distribution"].sum()
print(sum_1)
print(sum_2)
print(round(navs["subfund_total_net_asset"].iloc[0] - sum_, 5))
print(sum_)

23299334.48197175
129990578.91641122
7761794.81
322393682.12


In [25]:
instruments = generator.data_bucket.get_instruments_by_index(idx= ALL, info=["asset_name", "market_value_fund"])
fake_lines = instruments.loc[instruments["asset_name"].str.contains("Currency hedging liability", regex=False)]
fake_lines["market_value_fund"].sum()
generator.data_bucket.instruments.drop(fake_lines.index, inplace=True)

In [23]:
#generator.data_bucket.get_processing_data("distribution").loc[fake_lines.index]
curr_liabl = generator.data_bucket.processing_data.loc[(fake_lines.index, "LU1280365476"), "distribution"].sum()
print(dist_vec.sum())
print(curr_liabl)
print(dist_vec.sum() - curr_liabl)
print(generator.data_bucket.get_shareclass_nav(info=['shareclass_total_net_asset_sf_curr']).iloc[0])

23299334.48197175
-1712331.0322312966
25011665.514203046
23861795.52
